# Jupyter Notebook Title: Exploring Genetic Variation with pysam and ChatGPT

## Learning Objectives:

By the end of this activity, you will be able to:

- Use pysam to read and query VCF files.

- Extract metadata about samples and genotypes.

- Compute summary statistics like minor allele frequency (MAF) and Hardy-Weinberg equilibrium (HWE).

- Visualize genetic data using histograms.

- Use ChatGPT to guide problem-solving and code exploration.

## Getting Started

### VCF File Upload

In the Canvas activity page - I provided a VCF file. You'll need to upload that to your browser Jupyter environment. Above the file browser, you'll see an 'up' arrow that you can press to upload the file ending in `.vcf.gz` and the file ending in `.vcf.gz.tbi`

### Python Setup

In this activity - I'm going to direct you on what code to run to use the following packages that have been pre-installed into your environment. 

Those include: 

`pysam`, which is a python library intended for analyzing common/standard sequencing and variant file formats. We will use it a lot though this course. If you want to dig in to the technical documentation, that is here: https://pysam.readthedocs.io/en/latest/api.html

`numpy`, a library for working with numbers, arrays, etc... One of the most important scientific computing libraries in Python (https://numpy.org/)

`scipy`, lots of scientific algorithms. We are going to use it for some statistical methods (https://scipy.org/)


In [ ]:
# import our libraries

from pysam import VariantFile # pysam is a library for reading and writing SAM/BAM/CRAM files, and VariantFile is used for VCF/BCF files
import numpy as np # common alias for numpy is np
from scipy.stats import chisquare # in this case, we only want the chisquare function from scipy.stats module
from collections import Counter # this is part of the python standard library, but we need to import it

## Opening a VCF File

In the directory - you should see your uploaded files `kgenomes.cyp2c19.vcf.gz` and a similar one called `kgenomes.cyp2c19.vcf.gz.tbi`

The `.vcf.gz` is a *compressed* VCF file, and the `.tbi` file is the index for that file. It is indexed so the computer can open a specific part of the file without decompressing the whole thing. 

`pysam` (and many other tools) automatically recognize the index file. It just needs to be saved along with the main file. 

Check out the next code cell - you can run it as is to load the VCF file with pysam.VariantFile

In [ ]:
vcf_file = VariantFile("kgenomes.cyp2c19.vcf.gz")

# we can print the header with the following command: 
print(vcf_file.header)

## View Number of Samples in the VCF File

VCF files can support anywhere from zero to millions+ samples. 

Let's use a basic pysam method to view the number of samples in our VCF file. 

Ask ChatGPT for help with this one. Something like: *How can I view the number of samples in a VCF file with pysam?*

You might need to modify its output a bit. Note that it might also give you some information that is redundant with what we have already done. 

In [ ]:
# view number of samples here

## Calculate Minor Allele Frequencies

In the next code cells, I've defined for you a *function*.

This is a common tool in python for creating modular processes. 

Just like a mathematical function $f(x)=x+1$, a function takes one or more arguments (i.e. $x$) and does something to them.

Writing functions can be tricky. Especially when you're coupling them with VCF file parsing. The first one is for calculating the minor allele frequency from a single variant in a VCF file. 


In [ ]:
def calculate_maf(record):
    genotypes = [call['GT'] for call in record.samples.values()]
    alleles = [allele for gt in genotypes if gt is not None for allele in gt if allele is not None and allele >= 0]
    counts = Counter(alleles)
    total_alleles = sum(counts.values())
    minor_count = sorted(counts.values())[0]
    return minor_count / total_alleles

# feel free to copy and paste this function into ChatGPT to ask how it works. 

Recall that in a VCF file - every row is a variant. 
Let's calculate MAF for each row with our `calculate_maf` function. 

Run the following cell:

In [ ]:
vcf_file.reset()  # Reset the iterator to the beginning of the VCF file

# if you don't reset, the iterator will be at the end of the file after the first loop and it won't return any records in the second loop.

mafs = [calculate_maf(record) for record in vcf_file]

print(f"Calculated MAF for {len(mafs)} variants.")

To visualize the minor allele frequencies, ask ChatGPT something like *How can I make a histogram from a python list of numbers and matplotlib?*

## Calculate Hardy-Weinberg Equilibrium

A key component of population genetics quality control is HWE deviation. 

You can use the following function to calculate it from pysam records. 

Use the same approach we used for calculate MAF to also get HWE P values. 

*See if you can figure it out without using ChatGPT, but feel free to ask it if you get stuck.*

In [ ]:
def hardy_weinberg(record):
    genotypes = [call['GT'] for call in record.samples.values()]
    gt_counts = Counter(gt for gt in genotypes if None not in gt)
    
    obs = [gt_counts.get((0, 0), 0), gt_counts.get((0, 1), 0) + gt_counts.get((1, 0), 0), gt_counts.get((1, 1), 0)]
    n = sum(obs)
    if n == 0:
        return None, None
    
    p = (2 * obs[0] + obs[1]) / (2 * n)
    q = 1 - p
    
    exp = [p ** 2 * n, 2 * p * q * n, q ** 2 * n]
    
    chi2, pval = chisquare(obs, f_exp=exp)
    return pval

**Answer the following**: why do we use chi-squared to test for deviation from Hardy-Weinberg equilibrium?

Then - consider what you might do with the results. You have an array of p values. What defines a significant P value for HWE? How does that inform our next steps?

Hint - you might want to define a strict cutoff of 0.05 and count how many items in the list of HWE results are below that value. 

# Querying a Specific Variant

The data in our VCF file is taken from a public resource called the "1000 Genomes Project"

The variants are from a region on chromsome 10 that contains *CYP2C19*

-----

As an example - let's look at a single variant in *CYP2C19* that is associated with the \*2 allele.

This variant is at position 94781859 on chromosome 10. 

Check out the following code cell to see how you can do that. 

You can read more information about *CYP2C19* variants and star alleles here: https://www.pharmvar.org/gene/CYP2C19

In [ ]:
vcf_file.reset()  # Reset the iterator to the beginning of the VCF file

variant_records = vcf_file.fetch("10", 94781859 - 1, 94781859) # we fetch based on 0-based coordinates, so we subtract 1 from the position

# multiple variants might overlap with the position, so we need to iterate through the 
# records in our target region and just keep the one that matches our position exactly

variant_record = [record for record in variant_records if record.pos == 94781859][0]  # Get the first record from the iterator

# now that we have our variant record, we can calculate the MAF and HWE p-value

maf = calculate_maf(variant_record)
print(f"MAF for variant {variant_record.id} is {maf:.4f}")
pval = hardy_weinberg(variant_record)
print(f"HWE p-value for variant {variant_record.id} is {pval:.4e}")

## Reflection

1. What is a VCF file and how does it help us work with variant data?
2. How do functions and pysam make it easier to work with data in VCF files?

### Once you are finished - save + download this notebook and submit it through Canvas. 